In [1]:
from google.colab import drive
drive.mount('/content/drive/')

# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch
!pip install torchsummary

from PIL import Image
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import math;

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from torch.utils.data import Dataset, DataLoader,TensorDataset
from torchvision import transforms

from torchsummary import summary
%matplotlib inline

import os
os.chdir("/content/drive/My Drive/AML_KAGGLE")
os.getcwd()
os.listdir(os.getcwd())
#
#train_images = pickle.load(open(os.getcwd()+"/pkl/preprocessed_train_images.pkl", "rb"))
#train_labels = pickle.load(open(os.getcwd()+"/pkl/train_labels.pkl", "rb"))
#train_filenames = pickle.load(open(os.getcwd()+"/pkl/train_filenames.pkl", "rb"))
#test_images = pickle.load(open(os.getcwd()+"/pkl/preprocessed_test_images.pkl", "rb"))
#test_filenames = pickle.load(open(os.getcwd()+"/pkl/test_filenames.pkl", "rb"))
#
#

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/
tcmalloc: large alloc 1073750016 bytes == 0x584c0000 @  0x7f2c4952b2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


['Data', 'Deeper Network.ipynb']

In [0]:
import pickle
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import shuffle
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms

train_path = "/Users/dpap/Documents/UvA/AML/Assignments/Kaggle/pickle/initial_dataset/"
test_path = "/Users/dpap/Documents/UvA/AML/Assignments/Kaggle/pickle/initial_dataset/"

In [0]:
!ls pickle/initial_dataset/

ls: cannot access 'pickle/initial_dataset/': No such file or directory


In [0]:
os.getcwd()

'/content/drive/My Drive/AML_KAGGLE'

# Data

In [0]:
train_images = pickle.load(open(os.getcwd()+"/Data/processed_train_images.pkl","rb"))
train_labels = pickle.load(open(os.getcwd()+"/Data/processed_train_labels.pkl","rb"))

test_images  = pickle.load(open(os.getcwd()+"/Data/processed_test_images.pkl","rb"))
test_labels  = pickle.load(open(os.getcwd()+"/Data/processed_test_labels.pkl","rb"))

val_images   = pickle.load(open(os.getcwd()+"/Data/processed_val_images.pkl","rb"))
val_fname    = pickle.load(open(os.getcwd()+"/Data/processed_val_fnames.pkl","rb"))

In [0]:
val_images   = pickle.load(open(train_path+"val_images.pkl","rb"))
val_fname    = pickle.load(open(train_path+"val_fname.pkl","rb"))

In [0]:
print(len(test_images))
print(len(test_labels))

4883
4883


# Datasets and Dataloaders

In [0]:
class TrainingImagesDataset(Dataset):
    
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        single_image = self.images[index]
        single_label = self.labels[index]
        
        if self.transform != None:
            #image to tensor
            tensor = self.transform(single_image)
            return (tensor, single_label)
        #return image without turning it to tensor
        #probably wrong
        return (single_image, single_label)

In [0]:
class ValidationImagesDataset(Dataset):
    
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        single_image = self.images[index]
        if self.transform:
            #image to tensor
            tensor = self.transform(single_image)
            return tensor
        return single_image

In [0]:
def create_dataloaders(train_images, train_lbs, test_images=None, test_lbs = None, batch_size = 32):
    
    train_dataset = TrainingImagesDataset(train_images, train_lbs, transform=transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),transforms.RandomVerticalFlip(p=0.5),transforms.RandomRotation(degrees=360), transforms.ToTensor()]))
    
    if test_images and test_lbs:
        print("here")
        test_dataset = TrainingImagesDataset(test_images, test_lbs, transform=transforms.Compose([transforms.ToTensor()]))
    elif test_images:
        test_dataset = ValidationImagesDataset(test_images, transform=transforms.Compose([transforms.ToTensor()]))
    else:
        test_dataset = []
    
    train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 0)
    test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False, num_workers = 0)    
    
    return (train_loader, test_loader)

# Neural Network

In [0]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 12, 5, padding=2)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 32, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 48, 3, padding=1)
        #self.pool3 = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(48,32,3, padding=1)
        self.pool3 = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(2048, 2048)
        self.fc2 = nn.Linear(2048, 2048)
        self.fc3 = nn.Linear(2048, 121)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool3(F.relu(self.conv4(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


ocean = CNN().cuda()

In [0]:
torch.cuda.is_available()

True

# Training

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(ocean.parameters(), lr = 0.001, momentum = 0.9, weight_decay=0.0005)
lr_scheduler = ReduceLROnPlateau(optimizer, 'max', factor = 0.1, patience = 5, verbose=True)

train_loader, test_loader = create_dataloaders(train_images, train_labels, test_images, test_labels)

here


In [10]:
%%time
EPOCH = 25
for epoch in range(EPOCH):  # loop over the dataset multiple times
    ocean.train().cuda()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        # get the inputs
        inputs = inputs.cuda()
        lbs = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = ocean(inputs)
        loss = criterion(outputs, lbs)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0
    ocean.eval().cuda()
    correct = 0
    total = 0
    for (images, labels) in test_loader:
        #print("hi")
        images = images.cuda()
        labels = labels.cuda()
        #print(images)
        #input()
        outputs = ocean(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy on EPOCH %d test images: %d %%' % (epoch+1, 100 * correct / total))   
    lr_scheduler.step(correct / total)
print('Finished Training')

[1,  1000] loss: 0.213
[1,  2000] loss: 0.228
Accuracy on EPOCH 1 test images: 61 %
Epoch    50: reducing learning rate of group 0 to 1.0000e-04.
[2,  1000] loss: 0.125
[2,  2000] loss: 0.100
Accuracy on EPOCH 2 test images: 63 %
[3,  1000] loss: 0.085
[3,  2000] loss: 0.081
Accuracy on EPOCH 3 test images: 63 %
[4,  1000] loss: 0.072
[4,  2000] loss: 0.071
Accuracy on EPOCH 4 test images: 63 %
[5,  1000] loss: 0.067
[5,  2000] loss: 0.067
Accuracy on EPOCH 5 test images: 63 %
[6,  1000] loss: 0.063
[6,  2000] loss: 0.062
Accuracy on EPOCH 6 test images: 63 %
[7,  1000] loss: 0.060
[7,  2000] loss: 0.061
Accuracy on EPOCH 7 test images: 63 %
[8,  1000] loss: 0.057
[8,  2000] loss: 0.056
Accuracy on EPOCH 8 test images: 63 %
Epoch    57: reducing learning rate of group 0 to 1.0000e-05.
[9,  1000] loss: 0.055
[9,  2000] loss: 0.051
Accuracy on EPOCH 9 test images: 63 %
[10,  1000] loss: 0.053
[10,  2000] loss: 0.052
Accuracy on EPOCH 10 test images: 63 %
[11,  1000] loss: 0.051
[11,  200

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(ocean.parameters(), lr = 0.001, momentum = 0.9, weight_decay=0.0005)

In [0]:
train_loader, test_loader = create_dataloaders(train_images, train_labels, test_images, test_labels)

here


# Testing

In [0]:
correct = 0
total = 0
jj=0
with torch.no_grad():
    try:
        for (images, labels) in test_loader:
            try:
                jj+=1
                #print("hi")
                images = images.cuda()
                labels = labels.cuda()
                #print(images)
                #input()
                outputs = ocean(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                #print("here")
            except:
                print(jj)
    except:
        print(jj)
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
print(correct)
print(total)

Accuracy of the network on the 10000 test images: 59 %
2885
4883


In [0]:
class_correct = list(0. for i in range(121))
class_total = list(0. for i in range(121))
with torch.no_grad():
    for data in test_loader:
        images, lbs = data
        outputs = ocean(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == lbs).squeeze()
        #for lb in lbs:
        #    if lb == 120:
        #        print("here")
        #        input()
        for i in range(len(lbs)):
            label = lbs[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(121):
    try:
        #if class_correct[i]:
        #    print (i)
        #    print(class_correct[i])
        #    print(class_total[i])
        print('Accuracy of %5s : %2d %%' % (
            i, 100 * class_correct[i] / class_total[i]))
    except:
        #print(e)
        print(i)
        print(class_correct[i])
        print(class_total[i])

Accuracy of     0 :  3 %
Accuracy of     1 : 17 %
Accuracy of     2 : 72 %
Accuracy of     3 : 69 %
Accuracy of     4 : 38 %
Accuracy of     5 : 86 %
Accuracy of     6 : 72 %
Accuracy of     7 : 75 %
Accuracy of     8 : 48 %
Accuracy of     9 :  0 %
Accuracy of    10 :  0 %
Accuracy of    11 : 70 %
Accuracy of    12 : 52 %
Accuracy of    13 :  0 %
Accuracy of    14 : 15 %
Accuracy of    15 : 25 %
Accuracy of    16 : 20 %
Accuracy of    17 : 42 %
Accuracy of    18 : 50 %
Accuracy of    19 : 37 %
Accuracy of    20 : 31 %
Accuracy of    21 : 55 %
Accuracy of    22 : 57 %
Accuracy of    23 : 31 %
Accuracy of    24 : 27 %
Accuracy of    25 : 42 %
Accuracy of    26 : 72 %
Accuracy of    27 : 67 %
Accuracy of    28 : 63 %
Accuracy of    29 : 11 %
Accuracy of    30 :  0 %
Accuracy of    31 : 45 %
Accuracy of    32 : 80 %
Accuracy of    33 :  0 %
Accuracy of    34 : 51 %
Accuracy of    35 : 12 %
Accuracy of    36 : 79 %
Accuracy of    37 :  0 %
Accuracy of    38 : 13 %
Accuracy of    39 : 21 %


# Prediction

In [0]:
val_dataset = ValidationImagesDataset(val_images, transform=transforms.Compose([transforms.ToTensor()]))
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False, num_workers = 0)

In [12]:
prediction = []
i=0
for images in val_loader:
    i+=1
    outputs = ocean(images.cuda())
    #print(outputs.size())
    #print(outputs[0])
    #input()
    _, predicted = torch.max(outputs.data, 1)
    prediction.extend(predicted.cpu().numpy())
print(i)

192


# Output

In [0]:
results_df = pd.DataFrame({'image': val_fname, 'class': prediction}, columns=['image', 'class'])
results_df.to_csv(os.getcwd()+"/Data/results/results_new_3.csv", sep = ',', index = False)